In [1]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [95]:
menu_df = pd.read_csv(os.getcwd()+'/data/preproc/main/menu.csv').drop(columns=['type','writer','body','time','hour'])
sold_out_df = pd.read_csv(os.getcwd()+'/data/preproc/main/soldout.csv').drop(columns=['type','writer','time','body'])
sold_out_df .rename(columns={'hour': 'soldout_time'}, inplace=True)

In [96]:
sold_out_df['soldout'] = sold_out_df['soldout'].apply(ast.literal_eval) # 리스트 타입으로 바꿔주기 
sold_out_df['soldout'] = sold_out_df['soldout'].apply(sorted) # sorting 
sold_out_df  = sold_out_df.explode('soldout')
sold_out_df['soldout']  = sold_out_df['soldout'].astype(int)
sold_out_df= sold_out_df.groupby(['post_no','soldout']).agg({ # 해당 코스에 대한 품절이 가장 먼저 나온 시간 기준
    'soldout_time': 'min'
}).reset_index()

### 코스 기반 품절 여부

In [116]:
# 품절 된 경우만 필터링 
menu_df_w_soldout=pd.merge(menu_df, sold_out_df, how='left',on='post_no') # 조인 
menu_df_w_soldout['course_no'] = menu_df_w_soldout['course_no'].apply(ast.literal_eval) # 리스트 타입으로 다시 인식 

filtered_df = menu_df_w_soldout[menu_df_w_soldout.apply(lambda row: row['soldout'] in row['course_no'], axis=1)] # course 값이 매칭되는 경우만 필터링
filtered_df.loc[:, 'course_no'] = filtered_df['course_no'].astype(str)

# groupby 후 필터링 적용
grouped_df = filtered_df.groupby(['post_no','course_no']).agg({
    'soldout': lambda x: sorted(x.astype(int)),
    'soldout_time': 'max' # 만약 동일 음식이 1,3 코스에서 진행된다면, 1,3 중 더 늦게 품절 코스의 품절 시간
}).reset_index()

# 해당 음식을 제공하는 코스가 모두 품절 된 경우만 (course_no와 soldout 값이 동일한 경우만 필터링)
filtered_grouped_df = grouped_df[grouped_df['course_no'] == grouped_df['soldout'].astype(str)].copy()
filtered_grouped_df.loc[:, 'is_soldout']  = True

In [125]:
fin_menu_df = pd.merge(menu_df, filtered_grouped_df, how='left',on=['post_no','course_no']) 
fin_menu_df.is_soldout = [False if is_soldout!=True else True for is_soldout in fin_menu_df['is_soldout']]
fin_menu_df.to_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv', sep=',' ,na_rep='NaN',index = False)

In [127]:
fin_menu_df[fin_menu_df.meal_time == 'lunch'].agg({
    'rating':'mean', 
    'userId':'count'})

,post_no,day,day_of_week,meal_time,course_no,course,course_kcal,course_protein,course_na,soldout,soldout_time,is_soldout
0,926901171,2023-05-03,Wednesday,lunch,[1],"투움바파스타,미니함박&데미소스,오이피클,탄산음료",956,36,1270.0,[1],NaN,True
1,926901171,2023-05-03,Wednesday,lunch,[2],"닭곰탕,잡곡밥,언양식불고기,쥐어채볶음,고들빼기무침",882,44,1245.0,NaN,NaN,False
2,926901172,2023-05-03,Wednesday,dinner,[1],"육감쫄면,우동국물,후리가케밥(소),갈비만두찜,꼬들단무지",942,58,1466.0,NaN,NaN,False
3,926901172,2023-05-03,Wednesday,dinner,[2],"차돌된장찌개,잡곡밥,한식잡채,고구마순들깨나물,청경채겉절이",631,37,1429.0,NaN,NaN,False
4,926901173,2023-05-04,Thursday,lunch,[1],"<제철식재>매실매실청돈육불고기,잡곡밥,얼큰콩나물국,느타리버섯채소볶음,양념깻잎지",699,30,1251.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1706,926901166,2023-04-28,Friday,lunch,[1],"열무비빔국수,사골파국,보리밥(소),손만두찜*초간장,오복지무침",944,33,1114.0,NaN,NaN,False
1707,926901166,2023-04-28,Friday,lunch,[2],"돼지국밥*새우젓,잡곡밥,두부양념조림,미나리오징어초무침,브로콜리통마늘볶음",767,42,1409.0,[2],NaN,True
1708,926901167,2023-04-28,Friday,dinner,[1],"소고기숙주덮밥,미소시루,야채튀김&칠리소스,락교무침",883,39,1491.0,NaN,NaN,False
1709,926901169,2023-05-02,Tuesday,lunch,[1],"돈육김치찌개,잡곡밥,도톰동그랑땡전,감자채볶음,풀무원조미김",757,58,1484.0,NaN,NaN,False
